# Pharmacy Data Enrichment
This notebook is for exploring how to best enrich data collected from the state database for additional insights. 

## Notebook Configuration

In [61]:
import pandas as pd
import re 
import numpy as np

# Set pandas to display more rows/columns for wider dataframes
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)     # Adjust rows
pd.set_option('display.width', 1000)       # Increase width for better visibility

## Load Pharmacy Data

In [62]:
pharmacies = pd.read_csv('attempt_19.csv')
pharmacies.head()

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,CEDARS-SINAI MEDICAL CENTER -4TH FLOOR MAIN
0,986 INFUSION PHARMACY #1 INC,LSC 101414,Sterile Compounding Pharmacy,CLEAR,"July 1, 2025",ANAHEIM,California,ORANGE,92807.0,NaN
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,"March 1, 2025",SALINAS,California,MONTEREY,93901.0,NaN
2,ACCREDO HEALTH GROUP INC,NSC 101280,Nonresident Sterile Compounding Pharmacy,CLEAR,"December 1, 2025",WARRENDALE,Pennsylvania,OUT OF STATE,15086.0,NaN
3,ACCREDO HEALTH GROUP INC,NSC 101279,Nonresident Sterile Compounding Pharmacy,CLEAR,"December 1, 2025",ORLANDO,Florida,OUT OF STATE,32822.0,NaN
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,"July 1, 2025",ESCONDIDO,California,SAN DIEGO,92025.0,NaN


## Enrich Pharmacy Categorization

- [y/n] - is this a government facility?
- [y/n] - is this a satellite location?
- [y/n] - is this an infusion center? 
- [y/n] - is this a hospital or medical center? 
- [y/n] - is research institute or lab
- [y/n] - is this strictly a veterinarian pharmacy? 
- [y/n] - provides Vitamin/Nutrition Therapy drugs
- [y/n] - provide GLP1 drugs (Semaglutide, Tirzepatide)

Available Specialities:
- specialtyCancer
- specialtyHomeInfusion
- specialtyNutrition
- 

In [63]:
#Drop rogue column from data collection:
pharmacies = pharmacies.drop(columns=['CEDARS-SINAI MEDICAL CENTER -4TH FLOOR MAIN'])

#Correct Zip Code Datatype
pharmacies['Zip'] = pharmacies['Zip'].fillna(0).astype(int).astype(str)

#
pharmacies['Expiration Date'] = pd.to_datetime(pharmacies['Expiration Date'], format='%B %d, %Y')


# Create an isGovernment column and populate it based on the License Type
pharmacies['isGovernment'] = pharmacies['License Type'].str.contains('Government', case=False, na=False)

# Create an isSatellite column and populate it based on the License Type
pharmacies['isSatellite'] = pharmacies['License Type'].str.contains('Satellite', case=False, na=False)

# Define a list of search terms
infusion_center_search_terms = ['INFUSION', 'HOSPICARE', 'INTEGRATED CARE SYSTEMS', 'IV LEAGUE', 'KABAFUSION', 'MARSHALL MEDICAL ONCOLOGY PHARMACY', 'MODEL DRUG', 'NUTRISHARE', 'OMNICARE', 'OPTION CARE', 'OPTUM', 'OSO HOME CARE', 'OWL REXALL DRUG', 'PLAZA HOME CARE', 'POLARIS PHARMACY SERVICES', 'PREFERRED EXCELLENT CARE', 'QUADRACARE', 'SCRIPTS', 'SHARP CENTRAL PHARMACY SERVICES', 'TRINITY CARE RX', 'VIVE RX']

# Create a regex pattern to match any of the search terms for infusion centers
pattern = '|'.join(infusion_center_search_terms)

# Update 'isInfusionCenter' based on keywords in License TYpe and Pharmacy Name
pharmacies['isInfusionCenter'] = (
    pharmacies['License Type'].str.contains('Infusion', case=False, na=False) |
    pharmacies['Pharmacy Name'].str.contains(pattern, case=True, na=False)
)
# List of keywords to check for in the 'License Type' column for Hospitals
keywords = [
    'HOSPITAL', 'MEDICAL CENTER', 'ADVENTIST HEALTH', 'MEMORIAL', 'CAMPUS', 
    'REGIONAL', 'MED CTR', 'KAISER', 'OF USC', 'REHABILITATION', 'CEDARS-SINAI', 'CITY OF HOPE', 'CANCER CENTER', 'HOSPTIAL', 'CLINIC', 'HSP','OUTPATIENT', 'HEALTH SERVICES', 'HEALTH CENTER', 'COUNTY ONCOLOGY', 'HEALTH SYSTEM', 'STANFORD HEALTH CARE', 'CENTER FOR', 'UC DAVIS', 'UCI HEALTH', 'RUHS', 'BOWYER PHARMACY', 'CARDINAL HEALTH', 'CEDARS-SANAI', 'CONVERSIO HEALTH', 'CRMC', 'ONCOLOGY CENTER', 'EMANATE HEALTH', 'CARE PAVILION', 'HOAG', 'HIGHLAND CARE PAVILLION', 'KAWEAH HEALTH', 'HEALTHCARE DISTRICT', 'LPCH', 'LUNDQUIST INSTITUTE', 'MOUNTAIN VIEW PHARMACEUTICALS', 'PENTEC HEALTH', 'PRIME HEALTHCARE', 'PROVIDENCE ONCOLOGY', 'RADY CHILDREN\'S', 'RIVER\'S EDGE', 'SCRIPPS', 'SHRINERS', 'SOLEO HEALTH INC', 'SONOMA SPECIALTY PHARMACY', 'SUTTER LAKESIDE', 'TODD CANCER PAVILION', 'CORRECTIONAL FACILITY']

# Create a pattern that matches any of the keywords
keywords_pattern = '|'.join([re.escape(keyword) for keyword in keywords])

# Create 'isHospital' column based on whether any of the keywords are present in 'Pharmacy Name'
pharmacies['isHospital'] = pharmacies['Pharmacy Name'].str.contains(keywords_pattern, case=False, regex=True, na=False)

isVeterinaryOnlyKeywords = ['PETNET PHARMACEUTICAL', 'PROLAB PHARMACY', 'SPECIALTY VETERINARY PHARMACY', 'WEDGEWOOD PHARMACY LLC']

#Create Boolean isVeterinary Only column from our list of known only vet pharmacies 
pharmacies['isVeterinaryOnly'] = pharmacies['Pharmacy Name'].isin(isVeterinaryOnlyKeywords)

# Declare pharmacies known to offer specialty products: 
specialty_dict = {
    'HRT' : ['A & O SPECIALITY PHARMACY', 'AMBROSIA HEALTHCARE', 'BOOTHWYN PHARMACY', 'CHEMIQUE PHARMACEUTICALS', 'ANAZAOHEALTH', 'ENOVEX', 'FIRESIDE', 'INNOVATIVE COMPOUNDING PHARMACY', 'LEE SILSBY COMPOUNDING PHARMACY', 'MEDSPECIALTIES COMPOUNDING PHARMACY', 'O\'BRIEN PHARMACY', 'PACIFIC COMPOUNDING PHARMACY AND CONSULTATIONS INC', 'STRIVE PHARMACY', 'TMC ACQUISITION LLC DBA TAILOR MADE COMPOUNDING', 'TORMED PHARMACY', 'VALOR COMPOUNDING PHARMACY'],
    'Sexual Health': ['A & O SPECIALITY PHARMACY', 'ANAZAOHEALTH', 'LEE SILSBY COMPOUNDING PHARMACY', 'MEDSPECIALTIES COMPOUNDING PHARMACY', 'RX UNLIMITED PHARMACY', 'STRIVE PHARMACY', ],
    'Dermatology': ['A & O SPECIALITY PHARMACY', 'ALERACARE PHARMACY', 'BIOPLUS SPECIALTY INFUSION', 'BOOTHWYN PHARMACY', 'ANAZAOHEALTH', 'ENVOEX', 'FIRESIDE', 'IMPRIMISRX', 'KabaFusion', 'LEE SILSBY COMPOUNDING PHARMACY', 'MEDSPECIALTIES COMPOUNDING PHARMACY', 'O\'BRIEN PHARMACY', 'OWL WESTERN PHARMACY', 'STRIVE PHARMACY', 'VALOR COMPOUNDING PHARMACY', 'VLS PHARMACY INC'],
    'Pain Management': ['A & O SPECIALITY PHARMACY', 'AMBROSIA HEALTHCARE', 'BASIC HOME INFUSION', 'BOOTHWYN PHARMACY', 'BROOKS HEALTH CARE', 'CHEMIQUE PHARMACEUTICALS', 'ENVOEX', 'FIRESIDE', 'HEALIX INFUSION', 'HOPEWELL PHARMACY', 'MARINER ADVANCED PHARMACY CORP', 'MEDSPECIALTIES COMPOUNDING PHARMACY', 'O\'BRIEN PHARMACY', 'VLS PHARMACY INC'],
    'Gastroenterology': ['A & O SPECIALITY PHARMACY', 'ALERACARE PHARMACY', 'BOOTHWYN PHARMACY', 'IV LEAGUE', 'LEE SILSBY COMPOUNDING PHARMACY', 'NORTHBAY SPECIALTY PHARMACY', 'O\'BRIEN PHARMACY', 'OWL WESTERN PHARMACY'],
    'Nutrition, Anti-Aging, General Wellness': ['CHEMIQUE PHARMACEUTICALS', 'ANAZAOHEALTH', 'CURE STAT RX', 'IMPRIMISRX', 'INFUSERVE', 'LEE SILSBY COMPOUNDING PHARMACY', 'PHARMALABS LLC', 'TMC ACQUISITION LLC DBA TAILOR MADE COMPOUNDING', 'UNIVERSITY COMPOUNDING PHARMACY', 'VLS PHARMACY INC'],
    'Oncology': ['ACCREDO HEALTH GROUP INC', 'ALERACARE PHARMACY', 'ALLIANCERX', 'AMERICAN RIVER INFUSION', 'AMERIPHARMA', 'BIOPLUS SPECIALTY INFUSION', 'CHEMIQUE PHARMACEUTICALS', 'COMPLETE INFUSION CARE', 'CURE STAT RX', 'IV LEAGUE', 'MARINER ADVANCED PHARMACY CORP', 'NORTHBAY SPECIALTY PHARMACY', 'OWL WESTERN PHARMACY', 'PACIFIC COMPOUNDING PHARMACY AND CONSULTATIONS INC'],
    'Blood & Bleeding Disorders': ['ACCREDO HEALTH GROUP INC', 'ALERACARE PHARMACY', 'ALLIANCERX', 'AMERIPHARMA', 'BIOPLUS SPECIALTY INFUSION', 'BIOSCRIP INFUSION SERVICES', 'CHEMIQUE PHARMACEUTICALS', 'CURE STAT RX', 'OWL WESTERN PHARMACY'],
    'Hospice & Long Term Care': ['ADVANCE CARE PHARMACY', 'AMBROSIA HEALTHCARE', 'AMBROSIA HEALTHCARE', 'RED ROCK PHARMACY', 'CITRUS PHARMACY', 'DEl\'S PHARMACY', 'DIAMOND DRUGS', 'FIRESIDE', 'HYGEIA', 'IV LEAGUE', 'PACIFIC WEST PHARMACY INC', 'PHILLIPS RANCH PHARMACY', 'SKILLED NURSING PHARMACY', 'STAR PHARMACY', 'VERDUGO PLAZA PHARMACY ALHAMBRA'],
    'Cardiology': ['ALERACARE PHARMACY', 'BIOSCRIP INFUSION SERVICES', 'CENTRAL ADMIXTURE PHARMACY'],
    'Infectious Disease' : ['ALERACARE PHARMACY', 'AMERICAN RIVER INFUSION', 'BIOSCRIP INFUSION SERVICES', 'BROOKS HEALTH CARE', 'COMPLETE INFUSION CARE', 'DEl\'S PHARMACY', 'IV LEAGUE', 'MARINER ADVANCED PHARMACY CORP', 'OWL WESTERN PHARMACY'],
    'Low Molecular': ['ALERACARE PHARMACY', ],
    'Neurology': ['ALERACARE PHARMACY', 'ALLIANCERX', 'BIOSCRIP INFUSION SERVICES', 'KabaFusion', 'NORTHBAY SPECIALTY PHARMACY'],
    'Opthalmic': ['ALERACARE PHARMACY', 'BOOTHWYN PHARMACY', 'CURE STAT RX', 'IMPRIMISRX', 'O\'BRIEN PHARMACY', 'OSRX INC', 'TORMED PHARMACY', 'VALOR COMPOUNDING PHARMACY'],
    'Osteo': ['ALERACARE PHARMACY', 'BIOPLUS SPECIALTY INFUSION'],
    'Rheumatology': ['ALERACARE PHARMACY', 'ALLIANCERX', 'AMERICAN OUTCOMES MANAGEMENT', 'AMERICAN RIVER INFUSION', 'BIOPLUS SPECIALTY INFUSION', 'BIOSCRIP INFUSION SERVICES', 'HEALIX INFUSION', 'NORTHBAY SPECIALTY PHARMACY', 'OWL WESTERN PHARMACY'],
    'Urology' : ['ALERACARE PHARMACY', 'BIOPLUS SPECIALTY INFUSION', 'BIOPLUS SPECIALTY INFUSION', 'O\'BRIEN PHARMACY', 'PHARMALABS LLC', 'TORMED PHARMACY'],
    'Pulmonary' : ['ALLIANCERX', ],
    'Reproductive' : ['ALLIANCERX', 'BIOSCRIP INFUSION SERVICES', 'CURE STAT RX', 'FREEDOM FERTILITY PHARMACY'],
    'Veterinary' : ['AMBROSIA HEALTHCARE', 'BCP VETERINARY', 'BOOTHWYN PHARMACY', 'CHEMIQUE PHARMACEUTICALS', 'COVETRUS', 'CURE STAT RX', 'ENVOEX', 'FIRESIDE', 'HOPEWELL PHARMACY', 'LEE SILSBY COMPOUNDING PHARMACY', 'MEDSPECIALTIES COMPOUNDING PHARMACY', 'PETNET PHARMACEUTICAL', 'PROLAB PHARMACY', 'SPECIALTY VETERINARY PHARMACY', 'STRIVE PHARMACY', 'VALOR COMPOUNDING PHARMACY'],
    'Autoimmune Disease': ['AMERICAN OUTCOMES MANAGEMENT', 'AMERICAN RIVER INFUSION', 'BIOPLUS SPECIALTY INFUSION', 'HEALIX INFUSION', 'KabaFusion', 'MARINER ADVANCED PHARMACY CORP'],
    'TPN' : ['AMERIPHARMA', 'BROOKS HEALTH CARE', 'CENTRAL ADMIXTURE PHARMACY', 'CHEMIQUE PHARMACEUTICALS', 'COMPLETE INFUSION CARE', 'CURE STAT RX', 'DCRX INFUSION', 'DEl\'S PHARMACY', 'IV LEAGUE', 'KabaFusion', 'MARINER ADVANCED PHARMACY CORP'],
    'Pediatric' : ['BIOPLUS SPECIALTY INFUSION', 'BOOTHWYN PHARMACY', 'CORAM CSV', 'CURE STAT RX', 'ENVOEX', 'FIRESIDE', 'IV LEAGUE', 'O\'BRIEN PHARMACY', 'VLS PHARMACY INC'],
    'Genetic Disorders': ['BIOPLUS SPECIALTY INFUSION'],
    'Weight Loss': ['BOOTHWYN PHARMACY', 'RED ROCK PHARMACY', 'ANAZAOHEALTH', 'FIRESIDE', 'INNOVATIVE COMPOUNDING PHARMACY', 'PHARMALABS LLC', 'STRIVE PHARMACY', 'UNIVERSITY COMPOUNDING PHARMACY', 'VALOR COMPOUNDING PHARMACY'],
    'Hair Care': ['ANAZAOHEALTH'],
    'Dental' : ['FIRESIDE', 'LEE SILSBY COMPOUNDING PHARMACY', 'O\'BRIEN PHARMACY', 'VALOR COMPOUNDING PHARMACY'],
    'Ear, Nose, Throat': ['FIRESIDE', 'HOPEWELL PHARMACY', 'O\'BRIEN PHARMACY'],
    'Podiatry' : ['FIRESIDE'],
    'Nuclear Radiopharmacy' : ['ISORX RADIOPHARMACY', 'JUBILANT DRAXIMAGE INC DBA JUBILANT RADIOPHARMA', 'RLS USA INC', 'SOFIE CO']
    }

conditions_dictionary = {
    'Multiple Sclerosis': ['ACCREDO HEALTH GROUP INC', 'ALERACARE PHARMACY', 'ALLIANCERX', 'BIOPLUS SPECIALTY INFUSION', 'NORTHBAY SPECIALTY PHARMACY', 'OWL WESTERN PHARMACY'],
    'HIV/AIDS' : ['ACCREDO HEALTH GROUP INC', 'BIOPLUS SPECIALTY INFUSION', 'NORTHBAY SPECIALTY PHARMACY', 'OWL WESTERN PHARMACY', 'RX UNLIMITED PHARMACY'],
    'Crohns' : ['ALERACARE PHARMACY', 'BIOPLUS SPECIALTY INFUSION', 'LEE SILSBY COMPOUNDING PHARMACY'],
    'Cystic Fibrosis': ['ALERACARE PHARMACY', ],
    'Endometriosis': ['ALERACARE PHARMACY', ],
    'Autism': ['HOPEWELL PHARMACY', 'LEE SILSBY COMPOUNDING PHARMACY'],
    'Hepatitis': ['ACCREDO HEALTH GROUP INC', 'ALERACARE PHARMACY', 'ALLIANCERX', 'BIOPLUS SPECIALTY INFUSION', 'NORTHBAY SPECIALTY PHARMACY', 'OWL WESTERN PHARMACY'],
    'Addiction' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Chronic Fatigue Syndrome' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Erectile Dysfunction' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Fibromyalgia' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Fissures' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Hirsutism' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Infertility' : ['LEE SILSBY COMPOUNDING PHARMACY', 'FREEDOM FERTILITY PHARMACY'],
    'Lyme Disease' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Methylation Dysfunction' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Oral Mucositis' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'PANDAS' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Rosacea' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Shingles' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Skin Lightening' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Vaginosis' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Warts' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Wound Care' : ['LEE SILSBY COMPOUNDING PHARMACY'],
    'Organ Transplants' : ['AMERICAN OUTCOMES MANAGEMENT', 'BIOPLUS SPECIALTY INFUSION', 'CORAM CSV', 'NORTHBAY SPECIALTY PHARMACY']
}

In [64]:
# Review Pharmacies that are NOT a hospital currently 
available_pharmacies = pharmacies[(pharmacies['isHospital'] == False) & (pharmacies['isInfusionCenter'] == False) & (pharmacies['isGovernment'] == False )]


# Loop through the specialty_dict to create new columns in available_pharmacies
for specialty, pharmacies in specialty_dict.items():
    # Create a new column with camel case naming like isHRT, isSexualHealth, etc.
    column_name = f'is{specialty.replace(" ", "")}'  # Removes spaces and adds 'is' at the beginning
    available_pharmacies[column_name] = available_pharmacies['Pharmacy Name'].isin(pharmacies)
    
# Loop through the conditions_dictionary to create new columns in available_pharmacies
for condition, pharmacies in conditions_dictionary.items():
    # Create a new column with "services" as the prefix and camel case naming
    column_name = f'services{condition.replace(" ", "").replace("/", "")}'  # Removes spaces and slashes and adds 'services' at the beginning
    available_pharmacies[column_name] = available_pharmacies['Pharmacy Name'].isin(pharmacies)

available_pharmacies

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,isGovernment,isSatellite,isInfusionCenter,isHospital,isVeterinaryOnly,isHRT,isSexualHealth,isDermatology,isPainManagement,isGastroenterology,"isNutrition,Anti-Aging,GeneralWellness",isOncology,isBlood&BleedingDisorders,isHospice&LongTermCare,isCardiology,isInfectiousDisease,isLowMolecular,isNeurology,isOpthalmic,isOsteo,isRheumatology,isUrology,isPulmonary,isReproductive,isVeterinary,isAutoimmuneDisease,isTPN,isPediatric,isGeneticDisorders,isWeightLoss,isHairCare,isDental,"isEar,Nose,Throat",isPodiatry,isNuclearRadiopharmacy,servicesMultipleSclerosis,servicesHIVAIDS,servicesCrohns,servicesCysticFibrosis,servicesEndometriosis,servicesAutism,servicesHepatitis,servicesAddiction,servicesChronicFatigueSyndrome,servicesErectileDysfunction,servicesFibromyalgia,servicesFissures,servicesHirsutism,servicesInfertility,servicesLymeDisease,servicesMethylationDysfunction,servicesOralMucositis,servicesPANDAS,servicesRosacea,servicesShingles,servicesSkinLightening,servicesVaginosis,servicesWarts,servicesWoundCare,servicesOrganTransplants
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,2025-03-01,SALINAS,California,MONTEREY,93901,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,ACCREDO HEALTH GROUP INC,NSC 101280,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,WARRENDALE,Pennsylvania,OUT OF STATE,15086,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,ACCREDO HEALTH GROUP INC,NSC 101279,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,ORLANDO,Florida,OUT OF STATE,32822,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ESCONDIDO,California,SAN DIEGO,92025,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
29,ALERACARE PHARMACY,NSC 101806,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-11-01,PHOENIX,Arizona,OUT OF STATE,85032,False,False,False,False,False,False,False,True,False,True,False,True,True,False,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,WEDGEWOOD PHARMACY LLC,NSC 101918,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-04-01,SCOTTSDALE,Arizona,OUT OF STATE,85251,False,False,False,False,True,False,False,False,Fa